<a href="https://colab.research.google.com/github/bbr32/nlp_proj/blob/master/NLP_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.callbacks import Callback
import tensorflow_hub as hub
import tensorflow as tf
import re

from keras import backend as K
import keras.layers as layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, Embedding, Flatten, Activation, SpatialDropout1D
from keras.layers import Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model
from keras.utils import np_utils
from keras.engine import Layer
from keras import initializers, regularizers, constraints
from keras.layers import *

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, CuDNNGRU, CuDNNLSTM, Add, Reshape
from keras.layers import MaxPooling1D, Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from nltk.tokenize import sent_tokenize, word_tokenize

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'


import re
import math
# set seed
np.random.seed(123)

Using TensorFlow backend.


In [2]:
!wget https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_test_no_labels.csv
!wget https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_train.csv

--2019-12-20 16:22:27--  https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_test_no_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3537628 (3.4M) [text/plain]
Saving to: ‘dataset_test_no_labels.csv’

dataset_test_no_lab 100%[===================>]   3.37M  --.-KB/s    in 0.03s   

2019-12-20 16:22:28 (113 MB/s) - ‘dataset_test_no_labels.csv’ saved [3537628/3537628]

--2019-12-20 16:22:31--  https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 745463

In [3]:
df = pd.read_csv('dataset_train.csv', sep='\t')
df = df.drop(columns="index")
df.head(10)

,sentence_1,sentence_2,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,neutral
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,entailment
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,entailment
3,How do you know? All this is their information...,This information belongs to them.,entailment
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,neutral
5,my walkman broke so i'm upset now i just have ...,I'm upset that my walkman broke and now I have...,entailment
6,But a few Christian mosaics survive above the ...,Most of the Christian mosaics were destroyed b...,neutral
7,(Read for Slate 's take on Jackson's findings.),Slate had an opinion on Jackson's findings.,entailment
8,Gays and lesbians.,Heterosexuals.,contradiction
9,At the end of Rue des Francs-Bourgeois is what...,Place des Vosges is constructed entirely of gr...,contradiction


In [4]:
df_test = pd.read_csv('dataset_test_no_labels.csv', sep='\t')
test = df_test.drop(columns="index")
test.head()

,sentence_1,sentence_2
0,One senior FAA air traffic control manager sai...,A senior FFA air traffic control manager said ...
1,"Once constructed, this persisting psychologica...",Once unique experiences become personally mean...
2,Help us continue to offer the finest professio...,Support us to keep offering the best professio...
3,"At the end of the Wars of Spanish, Austrian, a...",Northern Italy was not easily given up to the ...
4,Update on the Democratic fund-raising scandal ...,Clinton said the agents had not told him anyth...


In [0]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1)

train, val = train_test_split(train, test_size=0.1)

In [0]:
def get_rnn_data(df):
    x = {
        'sentence1': df["sentence_1"],
        #
        'sentence2': df["sentence_2"],
        }
    return x

le = LabelEncoder()

X_train = get_rnn_data(train)
Y_train = np_utils.to_categorical(le.fit_transform(train["label"].values)).astype("int64")

X_val = get_rnn_data(val)
Y_val = np_utils.to_categorical(le.fit_transform(val["label"].values)).astype("int64")

X_test = get_rnn_data(test)
Y_test = np_utils.to_categorical(le.fit_transform(test["label"].values)).astype("int64")

In [14]:
print(X_val)

{'sentence1': 249097    and i should take off but uh i appreciate that...
194433    no we've only well we've been in Texas for two...
139418                                  Where are we going?
346904                                  i enjoyed it higher
21279     La Villette ' in the northeast corner of the c...
                                ...                        
194229    While Podhoretz may be correct in his opinion ...
142899    A former Confucian temple is presently the hom...
58062     However, we recognize that data and modeling l...
77836     When ah! I heard that, I arranged for her to b...
8310      From the ticket office you enter the temple co...
Name: sentence_1, Length: 35340, dtype: object, 'sentence2': 249097    I should leave now, but I am motivated to go t...
194433                We've been in Texas over three years.
139418                                  We are going where?
346904                                          I hated it.
21279     La Villette is 

In [0]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)
    
#     def get_config(self):
#         config = {'output_dim': self.output_dim}
    
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape
    
#     def get_config(self):
#         config = {'output_dim': self.output_dim}
        
custom_ob={'ElmoEmbeddingLayer': ElmoEmbeddingLayer, 'NonMasking': NonMasking}

In [8]:
def get_model():
    model = Sequential()
    inp1 = Input(shape=(1,), dtype="string", name="sentence1")
    inp2 = Input(shape=(1,), dtype="string", name="sentence2")
    
    def emb_layer(inp, col):
        x = ElmoEmbeddingLayer()(inp)
        return x

    x = concatenate([
                    emb_layer(inp1,"sen_1"),
                    emb_layer(inp2,"sen_2"),
                     ])
    
    x = NonMasking()(x)
    x = Reshape((1, 1024*2), input_shape=(1024*2,))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)
    x = Bidirectional(LSTM(128, return_sequences=True,recurrent_dropout=0.2))(x)

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    outp = Dense(3, activation="softmax", name="final_output")(x)
    
    model = Model(inputs=[inp1,inp2], outputs=outp)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'],
                 )

    return model

model = get_model()

model.summary()



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence1 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
sentence2 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
elmo_embedding_layer_1 (ElmoEmb (None, 1024)         4           sentence1[0][0]                  
__________________________________________________________________________________________________
elmo_embedding_layer_2 (ElmoEmb (None, 1024)         4           sentence2[0][0]                  
____________________________________________________________________________________________

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5,
                                            min_lr=0.00001)
file_path="checkpoint_SNLI_weights.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
early = EarlyStopping(monitor="val_acc", mode="max", patience=1)

model_callbacks = [checkpoint, early, learning_rate_reduction]

In [18]:
model.fit(X_train, Y_train,
          batch_size=32,
          epochs=2,
          verbose=1,
          validation_data=(X_val, Y_val),
          callbacks = model_callbacks
         )

Train on 318055 samples, validate on 35340 samples
Epoch 1/2
 55456/318055 [====>.........................] - ETA: 54:13 - loss: 1.0289 - acc: 0.4490

KeyboardInterrupt: ignored

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
model = load_model("/content/drive/My Drive/Colab Notebooks/NLP Projet/SNLI_model.h5", custom_objects=custom_ob)
#model.load_weights('/content/drive/My Drive/Colab Notebooks/NLP Projet/SNLI_weights.hdf5')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ValueError: ignored

In [0]:
test_pred = model.predict(X_test, batch_size=128)

In [0]:
test_acc = (np.argmax(test_pred, axis=1) == np.argmax(Y_test, axis=1)).sum()/Y_test.shape[0] * 100

print("Accuracy on test set is: %"+str(test_acc))